In [ ]:
# CAM Shift - Continuosly Adoptive Mean Shift, Object Tracking Camshift Method

# Mean Shift Object Tracking
# Object Tracking = is the process of locating a moving object over time using a camera. 

import cv2
import numpy as np

cap = cv2.VideoCapture("slow_traffic_small.mp4")

# take first frame of the video
ret, frame = cap.read()

# setup initial location of the window
x, y, w, h = 300, 200, 100, 50
track_window = (x, y, w, h)

# setup the ROI for tracking
roi = frame[y: y+h, x: x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask3 = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask3, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# setup the termination criteria, either 10 iteration or move by atleast 1 point
term_crit = (cv2.TermCriteria_EPS | cv2.TermCriteria_COUNT, 10, 1)
cv2.imshow("ROI", roi)

while(1):
    ret, frame = cap.read()
    if ret == True:

        hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dsst = cv2.calcBackProject([hsv1], [0], roi_hist, [0, 180], 1)

        #  apply mean shift to get the new location
        ret, track_window = cv2.CamShift(dsst, track_window, term_crit)

        # draw it on image
        # x, y, w, h = track_window
        # final_img = cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 3)
        pts = cv2.boxPoints(ret)
        # print(pts)
        pts = np.uint0(pts)
        f_img = cv2.polylines(frame, [pts], True, (255, 255, 0), 3)

        cv2.imshow("Final DST", dsst)
        cv2.imshow("Final Frame", f_img)
        
        k = cv2.waitKey(50)
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()